In [1]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==

In [2]:
import sys
import logging  # 用于日志记录

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset
from torch.nn import ModuleList
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DebertaV2Config,
    DebertaV2Model,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding  # 用于动态填充批次
)
# 从特定模块导入内部类
from transformers.models.deberta_v2.modeling_deberta_v2 import DebertaV2Encoder

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        # 明确指定输出到 sys.stdout 或 sys.stderr，有时更可靠
        logging.StreamHandler(sys.stdout)
    ],
    force=True  # <--- 添加这个参数
)

# 获取 logger 实例
logger = logging.getLogger(__name__)


2025-04-26 10:40:02.526698: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745664002.701910      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745664002.756034      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import evaluate  # <--- 添加 evaluate 导入

metric = evaluate.load("accuracy")  # <--- 使用 evaluate.load


def compute_metrics(eval_pred):
    """计算评估指标"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [4]:
logger.info("加载 IMDB 数据集...")
raw_datasets = load_dataset("imdb")

# 教师模型路径和学生模型保存路径
teacher_model_path = '/kaggle/input/deberta-v3-base-finetuned-imdb/deberta-v3-base-finetuned-imdb'  # 假设已微调好
student_model_path = 'deberta-v3-base-student-pkd'  # 学生模型保存路径

# 加载与教师模型匹配的分词器
logger.info(f"加载分词器: {teacher_model_path}")
tokenizer = AutoTokenizer.from_pretrained(teacher_model_path)


def preprocess_function(examples):
    """对数据集进行分词"""
    # 截断或填充到最大长度 (如果需要，可以调整 max_length)
    return tokenizer(examples["text"], truncation=True, padding=False)  # Trainer 会处理批次内的填充


logger.info("对数据集进行分词...")
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

# 移除原始文本列，设置格式为 PyTorch 张量
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

# 准备训练集和测试集 (IMDB 默认只有 train 和 test)
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]  # 使用 test 集作为评估集

# 数据整理器，用于动态填充批次中的序列到最大长度
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

logger.info("数据集加载和预处理完成。")


2025-04-26 10:40:15,989 [INFO] 加载 IMDB 数据集...


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

2025-04-26 10:41:17,122 [INFO] 数据集加载和预处理完成。


In [5]:
logger.info(f"加载教师模型: {teacher_model_path}")
teacher_model = AutoModelForSequenceClassification.from_pretrained(teacher_model_path)
teacher_model.eval()  # 设置为评估模式


2025-04-26 10:41:17,136 [INFO] 加载教师模型: /kaggle/input/deberta-v3-base-finetuned-imdb/deberta-v3-base-finetuned-imdb


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [6]:
# (使用你提供的 create_student_and_copy_weights, copy_deberta_weights, copy_encoder_weights 函数)
def create_student_and_copy_weights(teacher_model):
    """
    创建学生模型并通过复制教师模型的部分权重来初始化。
    """
    teacher_config = teacher_model.config
    student_config_dict = teacher_config.to_dict()
    original_num_layers = student_config_dict.get('num_hidden_layers', 12)
    student_num_layers = original_num_layers // 2
    student_config_dict['num_hidden_layers'] = student_num_layers
    logger.info(f"教师模型层数: {original_num_layers}, 学生模型层数: {student_num_layers}")
    student_config = DebertaV2Config.from_dict(student_config_dict)
    student_model = type(teacher_model)(config=student_config)
    logger.info("\n开始复制权重...")
    copy_deberta_weights(teacher_model, student_model)
    return student_model


def copy_encoder_weights(teacher_encoder, student_encoder):
    """专门处理 Encoder 层的权重复制"""
    if not (hasattr(teacher_encoder, 'layer') and hasattr(student_encoder, 'layer')):
        logger.error("Error: Cannot find 'layer' attribute in teacher or student encoder.")
        return
    if not isinstance(teacher_encoder.layer, ModuleList) or not isinstance(student_encoder.layer, ModuleList):
        logger.error("Error: Encoder 'layer' attribute is not a ModuleList.")
        return
    teacher_layers = teacher_encoder.layer
    student_layers = student_encoder.layer
    teacher_num_layers = len(teacher_layers)
    student_num_layers = len(student_layers)
    if student_num_layers == 0:
        logger.warning("Warning: Student encoder has no layers. Skipping encoder copy.")
        return
    if teacher_num_layers < student_num_layers:
        logger.warning("Warning: Teacher has fewer layers than student. Copying available layers.")
        teacher_indices_to_copy = list(range(teacher_num_layers))
    else:
        step = teacher_num_layers // student_num_layers
        teacher_indices_to_copy = [i * step for i in range(student_num_layers)]
    logger.info(f"将复制教师 Encoder 层索引: {teacher_indices_to_copy} 到学生层 [0..{student_num_layers - 1}]")
    num_copied = 0
    for i, teacher_idx in enumerate(teacher_indices_to_copy):
        if i < student_num_layers and teacher_idx < teacher_num_layers:
            try:
                student_layers[i].load_state_dict(teacher_layers[teacher_idx].state_dict())
                num_copied += 1
            except Exception as e:
                logger.error(f"Error copying layer {teacher_idx} to {i}: {e}")
        else:
            logger.warning(
                f"Warning: Index out of bounds during encoder layer copy (i={i}, teacher_idx={teacher_idx}). Skipping.")
    logger.info(f"已复制 {num_copied} 个 Encoder 层。")
    # 复制其他 Encoder 组件... (LayerNorm, rel_embeddings, conv)
    if hasattr(teacher_encoder, 'layer_norm') and hasattr(student_encoder,
                                                          'layer_norm') and teacher_encoder.layer_norm is not None and student_encoder.layer_norm is not None:
        try:
            student_encoder.layer_norm.load_state_dict(teacher_encoder.layer_norm.state_dict()); logger.info(
                "已复制 Encoder 中的 LayerNorm。")
        except Exception as e:
            logger.error(f"Error copying encoder LayerNorm: {e}")
    if hasattr(teacher_encoder, 'rel_embeddings') and hasattr(student_encoder,
                                                              'rel_embeddings') and teacher_encoder.rel_embeddings is not None and student_encoder.rel_embeddings is not None:
        try:
            student_encoder.rel_embeddings.load_state_dict(teacher_encoder.rel_embeddings.state_dict()); logger.info(
                "已复制 Encoder 中的 Relative Embeddings。")
        except Exception as e:
            logger.error(f"Error copying relative embeddings: {e}")
    if hasattr(teacher_encoder, 'conv') and hasattr(student_encoder,
                                                    'conv') and teacher_encoder.conv is not None and student_encoder.conv is not None:
        try:
            student_encoder.conv.load_state_dict(teacher_encoder.conv.state_dict()); logger.info(
                "已复制 Encoder 中的 Conv layer (Disentangled Attention)。")
        except Exception as e:
            logger.error(f"Error copying conv layer: {e}")


def copy_deberta_weights(teacher, student):
    """递归地将 DeBERTa 模型的权重从 teacher 复制到 student。"""
    teacher_children = dict(teacher.named_children())
    student_children = dict(student.named_children())
    for name, teacher_child in teacher_children.items():
        if name in student_children:
            student_child = student_children[name]
            if isinstance(teacher_child, DebertaV2Encoder):
                logger.info(f"找到 Encoder 模块: '{name}'。正在调用 copy_encoder_weights...")
                copy_encoder_weights(teacher_child, student_child)
            elif isinstance(teacher_child, DebertaV2Model):
                logger.info(f"递归进入基础模型模块: '{name}'...")
                copy_deberta_weights(teacher_child, student_child)
            elif isinstance(teacher_child, torch.nn.Module) and hasattr(teacher_child, 'state_dict'):
                try:
                    student_child.load_state_dict(teacher_child.state_dict())
                    logger.info(f"已复制子模块权重: '{name}'")
                except RuntimeError as e:
                    logger.warning(f"无法直接复制 '{name}' 的 state_dict。错误: {e}。尝试递归...")
                    if len(list(teacher_child.children())) > 0:
                        copy_deberta_weights(teacher_child, student_child)
                    else:
                        logger.warning(f"跳过复制 '{name}' (无子模块且 state_dict 不匹配)。")
                except Exception as e:
                    logger.error(f"复制 '{name}' 时发生未知错误: {e}")
            elif isinstance(teacher_child, torch.nn.Module) and len(list(teacher_child.children())) > 0:
                logger.info(f"递归进入容器模块: '{name}'...")
                copy_deberta_weights(teacher_child, student_child)


# --- 执行创建和复制 ---
logger.info("创建并初始化学生模型...")
student_model = create_student_and_copy_weights(teacher_model)
logger.info("\n学生模型创建并初始化完成。")


2025-04-26 10:41:17,784 [INFO] 创建并初始化学生模型...
2025-04-26 10:41:17,785 [INFO] 教师模型层数: 12, 学生模型层数: 6
2025-04-26 10:41:19,982 [INFO] 
开始复制权重...
2025-04-26 10:41:19,982 [INFO] 递归进入基础模型模块: 'deberta'...
2025-04-26 10:41:21,813 [INFO] 已复制子模块权重: 'embeddings'
2025-04-26 10:41:21,814 [INFO] 找到 Encoder 模块: 'encoder'。正在调用 copy_encoder_weights...
2025-04-26 10:41:21,814 [INFO] 将复制教师 Encoder 层索引: [0, 2, 4, 6, 8, 10] 到学生层 [0..5]
2025-04-26 10:41:22,561 [INFO] 已复制 6 个 Encoder 层。
2025-04-26 10:41:22,572 [INFO] 已复制 Encoder 中的 Relative Embeddings。
2025-04-26 10:41:22,587 [INFO] 已复制子模块权重: 'pooler'
2025-04-26 10:41:22,588 [INFO] 已复制子模块权重: 'classifier'
2025-04-26 10:41:22,589 [INFO] 已复制子模块权重: 'dropout'
2025-04-26 10:41:22,589 [INFO] 
学生模型创建并初始化完成。


In [7]:
# (使用你提供的 distillation_loss, patience_loss, PatientDistillationTrainingArguments, PatientDistillationTrainer)
def distillation_loss(y, labels, teacher_scores, T, alpha, reduction_kd='batchmean', reduction_nll='mean'):
    """计算标准知识蒸馏损失 (KLDiv) 和硬标签损失 (CrossEntropy) 的加权和。"""
    if teacher_scores is not None:
        kl_loss_fn = nn.KLDivLoss(reduction=reduction_kd)
        d_loss = kl_loss_fn(F.log_softmax(y / T, dim=-1),
                            F.softmax(teacher_scores / T, dim=-1)) * (T * T)
    else:
        assert alpha == 0, f'alpha cannot be {alpha} when teacher scores are not provided'
        d_loss = torch.tensor(0.0).to(y.device)
    nll_loss = F.cross_entropy(y, labels, reduction=reduction_nll)
    tol_loss = alpha * d_loss + (1.0 - alpha) * nll_loss
    return tol_loss, d_loss, nll_loss


def patience_loss(teacher_patience, student_patience, normalized_patience=False):
    """计算教师和学生中间层表示之间的 MSE 损失。"""
    teacher_patience = teacher_patience.float()
    student_patience = student_patience.float()
    if normalized_patience:
        teacher_patience = F.normalize(teacher_patience, p=2, dim=-1)
        student_patience = F.normalize(student_patience, p=2, dim=-1)
    loss = F.mse_loss(student_patience, teacher_patience)
    return loss


class PatientDistillationTrainingArguments(TrainingArguments):
    """包含蒸馏和 PKD 参数的训练参数类"""

    def __init__(self, *args, alpha=0.5, temperature=2.0, beta=1.0, normalized_patience=False, **kwargs):
        super().__init__(*args, **kwargs)
        self.alpha = alpha
        self.temperature = temperature
        self.beta = beta
        self.normalized_patience = normalized_patience


class PatientDistillationTrainer(Trainer):
    """自定义 Trainer，实现 PKD 损失计算"""

    def __init__(self, *args, teacher_model=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model
        if self.teacher_model is not None:
            self._move_model_to_device(self.teacher_model, self.model.device)
            self.teacher_model.eval()
        # 确定层映射关系
        teacher_num_layers = self.teacher_model.config.num_hidden_layers if self.teacher_model else 0
        student_num_layers = self.model.config.num_hidden_layers
        self.teacher_layer_indices = []
        self.student_layer_indices = []
        if student_num_layers > 0 and teacher_num_layers >= student_num_layers:
            step = teacher_num_layers // student_num_layers
            self.teacher_layer_indices = [i * step for i in range(student_num_layers)]
            self.student_layer_indices = list(range(student_num_layers))
        logger.info(
            f"PKD: Matching Teacher Layers {self.teacher_layer_indices} with Student Layers {self.student_layer_indices}")

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        if not isinstance(self.args, PatientDistillationTrainingArguments):
            raise ValueError("Trainer must be initialized with PatientDistillationTrainingArguments")
        labels = inputs.pop("labels")  # Trainer 会自动处理标签移动到设备
        student_outputs = model(**inputs, output_hidden_states=True)
        student_logits = student_outputs.logits
        student_hidden_states = student_outputs.hidden_states
        # 教师模型计算
        teacher_logits = None
        teacher_hidden_states = None
        if self.teacher_model is not None:
            with torch.no_grad():
                teacher_outputs = self.teacher_model(**inputs, output_hidden_states=True)
                teacher_logits = teacher_outputs.logits
                teacher_hidden_states = teacher_outputs.hidden_states
        # 计算标准蒸馏损失 + 硬标签损失
        standard_total_loss, kd_loss, nll_loss = distillation_loss(
            y=student_logits, labels=labels, teacher_scores=teacher_logits,
            T=self.args.temperature, alpha=self.args.alpha
        )
        # 计算 PKD 损失
        pkd_loss = torch.tensor(0.0).to(model.device)
        num_matched_layers = 0
        if student_hidden_states is not None and teacher_hidden_states is not None and \
                len(self.student_layer_indices) > 0 and \
                len(student_hidden_states) > max(self.student_layer_indices, default=-1) + 1 and \
                len(teacher_hidden_states) > max(self.teacher_layer_indices, default=-1) + 1:
            for s_idx, t_idx in zip(self.student_layer_indices, self.teacher_layer_indices):
                student_patience = student_hidden_states[s_idx + 1]
                teacher_patience = teacher_hidden_states[t_idx + 1]
                if student_patience.shape == teacher_patience.shape:
                    layer_pkd_loss = patience_loss(
                        teacher_patience=teacher_patience, student_patience=student_patience,
                        normalized_patience=self.args.normalized_patience
                    )
                    pkd_loss += layer_pkd_loss
                    num_matched_layers += 1
                else:
                    logger.warning(
                        f"PKD Layer dimension mismatch: Student {s_idx + 1} ({student_patience.shape}) vs Teacher {t_idx + 1} ({teacher_patience.shape}). Skipping.")

            if num_matched_layers > 0: pkd_loss /= num_matched_layers
        elif len(self.student_layer_indices) > 0:
            logger.warning("Could not compute PKD loss. Hidden states not available or insufficient layers.")
        # 总损失
        total_loss = standard_total_loss + self.args.beta * pkd_loss
        # 返回结果
        # 创建一个类似 Trainer 输出的字典，包含损失和 logits
        output_dict = {"loss": total_loss, "logits": student_outputs.logits}
        return (total_loss, output_dict) if return_outputs else total_loss



In [8]:
logger.info("配置训练参数...")
training_args = PatientDistillationTrainingArguments(
    output_dir="./pkd_imdb_deberta_student_output",  # 输出目录
    # max_steps = 1,
    num_train_epochs=3,  # 训练轮数 (减少以快速演示)
    per_device_train_batch_size=1,  # 训练批次大小 (减小以适应显存)
    per_device_eval_batch_size=4,  # 评估批次大小
    gradient_accumulation_steps=32,  # 梯度累积
    learning_rate=5e-5,             # 学习率 (可调整)
    gradient_checkpointing=True,
    warmup_ratio=0.1,  # 预热比例
    weight_decay=0.01,  # 权重衰减
    logging_dir='./pkd_logs',  # 日志目录
    logging_strategy="steps",
    logging_steps=50,  # 每 N 步记录一次日志
    eval_strategy="epoch",  # 每轮结束后评估
    save_strategy="epoch",  # 每轮结束后保存模型
    load_best_model_at_end=True,  # 训练结束后加载最佳模型
    metric_for_best_model="accuracy",  # 选择最佳模型的指标
    greater_is_better=True,  # accuracy 越高越好
    fp16=torch.cuda.is_available(),  # 如果有 GPU，使用混合精度
    report_to="tensorboard",  # 可以将日志发送到 tensorboard
    # --- PKD 特定参数 ---
    alpha=0.5,  # 标准蒸馏损失权重 (0.5 表示蒸馏和硬标签各占一半)
    temperature=4.0,  # 蒸馏温度 (需要调整)
    beta=1.0,  # PKD 损失权重 (需要调整, 1.0 是一个起点)
    normalized_patience=False,  # 是否归一化中间层输出 (可以尝试 True 或 False)
)

logger.info("实例化 PatientDistillationTrainer...")
trainer = PatientDistillationTrainer(
    model=student_model,  # 初始化好的学生模型
    teacher_model=teacher_model,  # 加载好的教师模型
    args=training_args,  # 包含 PKD 参数的训练参数
    train_dataset=train_dataset,  # 训练数据集
    eval_dataset=eval_dataset,  # 评估数据集
    tokenizer=tokenizer,  # 分词器
    data_collator=data_collator,  # 数据整理器
    compute_metrics=compute_metrics,  # 评估指标计算函数
)


2025-04-26 10:41:22,631 [INFO] 配置训练参数...
2025-04-26 10:41:22,655 [INFO] 实例化 PatientDistillationTrainer...


/tmp/ipykernel_19/714878743.py:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `PatientDistillationTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


2025-04-26 10:41:24,624 [INFO] PKD: Matching Teacher Layers [0, 2, 4, 6, 8, 10] with Student Layers [0, 1, 2, 3, 4, 5]


In [9]:
logger.info("开始 Patient Knowledge Distillation 训练...")
try:
    train_result = trainer.train()

    # --- 保存最终模型和训练状态 ---
    logger.info(f"训练完成！保存最佳学生模型到: {student_model_path}")
    trainer.save_model(student_model_path)  # 保存最好的学生模型
    trainer.log_metrics("train", train_result.metrics)
    trainer.save_metrics("train", train_result.metrics)
    trainer.save_state()

    # --- 进行最终评估 ---
    logger.info("对测试集进行最终评估...")
    eval_metrics = trainer.evaluate(eval_dataset=eval_dataset)
    trainer.log_metrics("eval", eval_metrics)
    trainer.save_metrics("eval", eval_metrics)

except Exception as e:
    logger.error(f"训练过程中发生错误: {e}", exc_info=True)  # 打印详细错误信息

logger.info("脚本执行完毕。")

2025-04-26 10:41:24,641 [INFO] 开始 Patient Knowledge Distillation 训练...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.639800,0.556859,0.908640
2,0.411400,0.442378,0.926960


2025-04-26 13:40:35,640 [INFO] 训练完成！保存最佳学生模型到: deberta-v3-base-student-pkd
***** train metrics *****
  epoch                    =     2.9971
  total_flos               =  5497926GF
  train_loss               =     0.6664
  train_runtime            = 2:59:10.58
  train_samples_per_second =      6.976
  train_steps_per_second   =      0.218
2025-04-26 13:40:37,114 [INFO] 对测试集进行最终评估...


***** eval metrics *****
  epoch                   =     2.9971
  eval_accuracy           =      0.927
  eval_loss               =     0.4424
  eval_runtime            = 0:20:27.46
  eval_samples_per_second =     20.367
  eval_steps_per_second   =      5.092
2025-04-26 14:01:04,587 [INFO] 脚本执行完毕。
